In [1]:
import numpy as np, os

In [2]:
def MkDirs(t):
    try:
        os.makedirs(t)
    except:
        pass

In [3]:
x = np.load('preprocessed0.npy')

In [4]:
x.shape

(40021, 278)

In [5]:
x[:,1]

array([78., 78., 78., ..., 80., 80., 80.])

In [6]:
BINS = 'Bins/'
for i in range(17, 91, 5):
    MkDirs(BINS + str(i) + '-' + str(i + 4))

In [7]:
x[333].shape

(278,)

In [26]:
L = len(x)

def GetBinOff(age):
    age = min(max(age, 17), 91)
    age -= 17
    return int(age) // 5

l = [[] for i in range(15)]

for i in range(L):
    row = x[i]
    age = row[1]
    off = GetBinOff(age)
    l[off].append(row)

ins = [np.array(i) for i in l]

In [27]:
IN_END = '/in.npy'

In [37]:
def GetPath(i):
    a = 17 + i * 5
    b = a + 4
    path = str(a) + '-' + str(b)
    return path

for i in range(15):
    path = GetPath(i)
    np.save(BINS + path + IN_END, ins[i])

# KNN

In [29]:
from sklearn.neighbors import KDTree

In [30]:
A, B = 148, 275 # posições dos coeficientes da onda inteira

In [53]:
for i in range(12): # último 72-76 pq 87-91 tem mto pouco
    dataDst = np.load(BINS + GetPath(i + 2) + IN_END) # 10 anos à frente
    ondas = []
    mappings = []
    for rowOff in range(len(dataDst)):
        row = dataDst[rowOff]
        if np.all(row[2:8] == 0): # labels
            c = row[A:B]
            ondas.append(c)
            mappings.append(rowOff)
    ondas = np.array(ondas)
    tree = KDTree(ondas)

    outStart = BINS + GetPath(i)
    dataSrc = np.load(outStart + IN_END)
    out = []
    for row in dataSrc:
        onda = row[A:B]
        dist, ind = tree.query(onda.reshape(1, -1))
        gIdx = dist.argmin()
        idxClosestOnda = ind[gIdx][0]
        orgRow = mappings[idxClosestOnda]
        out.append(dataDst[orgRow])
    np.save(outStart + '/out.npy', out)